In [1]:
from glob import glob
import json
import random
from tqdm import tqdm

In [2]:
fopen_l = open('combine-malay-no-alignment-rag.jsonl', 'w')

In [3]:
rejected = [
    'help.openai.com',
    'openassistant'
]

break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'model bahasa AI',
    'mempunyai kepercayaan atau pendapat peribadi',
    'tidak mempunyai pendapat peribadi',
    'tidak mempunyai kepercayaan',
    'tidak mempunyai falsafah peribadi',
    'tidak mempunyai pengalaman peribadi',
    'tidak mempunyai pendapat atau pengalaman peribadi',
    'tidak mempunyai maklumat terkini',
    'tidak mempunyai emosi peribadi',
    'tidak mempunyai keutamaan',
    'saya tidak mempunyai akses',
    'tidak mempunyai pengalaman',
    'saya tidak mempunyai keupayaan',
    'tidak mempunyai keupayaan',
    'tidak mempunyai hubungan',
    'tidak mempunyai maklumat',
    'Saya tidak mempunyai',
    'Saya tidak pernah',
    'saya tidak dapat memahami jawapan',
]

rejected_words = [
    'kebutuhan',
    'berbeda',
    'bahwa',
    'Kode',
    'kode',
    'nomor',
    'RMXX,XXX',
    'kompleksitas',
    'listrik',
    'jawaban',
    'teknis',
    'berkualitas',
    'mencoba',
    'kampanye',
    'komunitas',
    'stabilitas',
    'Stabilitas',
    'metode',
    'pria',
    'butuh',
    'jadwal',
    'kasus',
    'otomatis',
    'populer',
    'bisnis',
    'probabilitas',
    'rusak',
    'kapasitas',
    'rutinitas',
    'pertama-tama',
    'bagian',
    'fungsionalitas',
]

def found_word(s, words):
    for i in range(len(words)):
        if words[i] in s:
            return True, words[i]
    return False, None

indons = []

def accept(d, min_len = 10, skip_indon = True):
    global indons
    
    if len(d.split()) < min_len:
        return False
        
    if 'terjemahkan teks' in d.lower():
        return False

    if 'no need to translate' in d.lower():
        return False

    if 'can be translated' in d.lower():
        return False

    if 'cannot translate' in d.lower():
        return False

    if 'should be translated to' in d.lower():
        return False

    if 'cannot be translated' in d.lower():
        return False

    if 'standard malay' in d.lower():
        return False

    if 'would not be translated' in d.lower():
        return False

    if 'as an AI language model' in d:
        return False

    if 'should be translated as' in d.lower():
        return False

    if 'Bahasa Malaysia Standard' in d:
        return False
    
    if 'Saya adalah model AI' in d:
        return False
    
    if 'saya model AI' in d:
        return False
    
    if 'Saya model AI' in d:
        return False
    
    if 'sebagai model AI' in d:
        return False
    
    if 'Sebagai model AI' in d:
        return False
    
    if 'model bahasa AI' in d:
        return False
    
    if 'model AI yang dibangunkan' in d:
        return False
    
    if d == '<s>':
        return False
    
    if 'tidak dapat memberikan maklumat' in d:
        return False
    
    if 'Sebagai model bahasa' in d:
        return False
    
    d_lower = d.lower()
    if found_word(d_lower, rejected)[0]:
        return False
    
    if found_word(d_lower, break_at)[0]:
        return False
    
    found_indon = found_word(d_lower, rejected_words)
    if skip_indon and found_indon[0]:
        indons.append((d, found_indon[1]))
        return False
    
    return True

In [4]:
options = ['A.', 'B.', 'C.', 'D.']
options2 = ['A', 'B', 'C', 'D']
def rejected_output(q, a):
    q_, a_ = [], []
    for o in options:
        q_.append(o in q)
    for o in options2:
        a_.append(o in a)
    return sum(a_) > 1 or sum(q_) < 4

In [5]:
files = [
    'prepared-chatgpt-malaysian-general-qa.jsonl',
    'prepared-chatgpt-malaysian-open-qa.jsonl',
    'prepared-chatgpt4-malaysian-general-qa.jsonl',
    'prepared-mixtral-malaysian-general-qa.jsonl',
]

for f in files:
    print(f)
    with open(f) as fopen:
        for l in fopen:
            try:
                data = json.loads(l)
                if not accept(data['input'], skip_indon = False):
                    continue
                if not accept(data['output'], min_len = 1):
                    continue
                fopen_l.write(f'{json.dumps(data)}\n')
                fopen_l.flush()
            except Exception as e:
                pass

prepared-chatgpt-malaysian-general-qa.jsonl
prepared-chatgpt-malaysian-open-qa.jsonl
prepared-chatgpt4-malaysian-general-qa.jsonl
prepared-mixtral-malaysian-general-qa.jsonl


In [6]:
files = [
    'prepared-kertas-1.jsonl',
    'prepared-synthetic-commonsense.jsonl',
    'prepared-kesalahan-tatabahasa.jsonl',
    'prepared-malaysian-qa-choice.jsonl',
]

for f in files:
    print(f)
    with open(f) as fopen:
        for l in fopen:
            try:
                data = json.loads(l)
                if not accept(data['input'], skip_indon = False):
                    continue
                if not len(data['output'].strip()):
                    continue
                if rejected_output(data['input'], data['output']):
                    continue
                fopen_l.write(f'{json.dumps(data)}\n')
                fopen_l.flush()
            except Exception as e:
                print(e)
                pass

prepared-kertas-1.jsonl
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
prepared-synthetic-commonsense.jsonl
prepared-kesalahan-tatabahasa.jsonl
prepared-malaysian-qa-choice.jsonl


In [7]:
files = [
    'prepared-chatgpt-malay-instructions.jsonl',
    'prepared-camel-ai.jsonl',
]

for f in files:
    print(f)
    with open(f) as fopen:
        for l in fopen:
            if '```' in l:
                continue
            try:
                data = json.loads(l)
                if not accept(data['input'], skip_indon = False):
                    continue
                if not accept(data['output']):
                    continue
                fopen_l.write(f'{json.dumps(data)}\n')
                fopen_l.flush()
            except:
                pass

prepared-chatgpt-malay-instructions.jsonl
prepared-camel-ai.jsonl


In [8]:
files = [
    'prepared-malaysian-ultrachat-nocode.jsonl',
    'prepared-mixtral-malaysian-multiturn.jsonl',
    'prepared-mixtral-malaysian-rag.jsonl',
    'prepared-mixtral-malaysian-rag-knowledge-base.jsonl',
]

for f in files:
    print(f)
    with open(f) as fopen:
        for l in fopen:

            if 'tolong terjemah' in l.lower():
                continue

            try:
                data = json.loads(l)
                if not accept(data['input']):
                    continue
                fopen_l.write(f'{json.dumps(data)}\n')
                fopen_l.flush()
            except:
                pass

prepared-malaysian-ultrachat-nocode.jsonl
prepared-mixtral-malaysian-multiturn.jsonl
prepared-mixtral-malaysian-rag.jsonl
prepared-mixtral-malaysian-rag-knowledge-base.jsonl


In [9]:
files = glob('prepared-ultrachat-*')
files = [f for f in files if 'mesolitica' not in f]

count = 0
for f in files:
    print(f)
    with open(f) as fopen:
        
        for l in fopen:
            
            if 'Saya mahir dalam bahasa' in l:
                continue
                
            if 'tolong terjemah' in l.lower():
                continue
                
            if '```' in l:
                continue
                    
            try:
                data = json.loads(l)
                if not accept(data['input']):
                    continue
                    
                if random.random() > 0.2:
                    continue
                    
                fopen_l.write(f'{json.dumps(data)}\n')
                fopen_l.flush()
                
                count += 1
            except:
                pass
            
count

prepared-ultrachat-train_8.jsonl.translated.malay.no-alignment
prepared-ultrachat-train_4.jsonl.translated.malay.no-alignment
prepared-ultrachat-train_9.jsonl.translated.malay.no-alignment
prepared-ultrachat-train_7.jsonl.translated.malay.no-alignment
prepared-ultrachat-train_1.jsonl.translated.malay.no-alignment
prepared-ultrachat-train_0.jsonl.translated.malay.no-alignment
prepared-ultrachat-train_6.jsonl.translated.malay.no-alignment
prepared-ultrachat-train_5.jsonl.translated.malay.no-alignment
prepared-ultrachat-train_3.jsonl.translated.malay.no-alignment


176868

In [10]:
with open('prepared-combine-multiple-chats-nocode.jsonl') as fopen:
    for l in fopen:

        if 'tolong terjemah' in l.lower():
            continue

        try:
            data = json.loads(l)
            if not accept(data['input']):
                continue
            fopen_l.write(f'{json.dumps(data)}\n')
            fopen_l.flush()
        except:
            pass

In [11]:
len(indons)

94954

In [12]:
!wc -l combine-malay-no-alignment-rag.jsonl

2775535 combine-malay-no-alignment-rag.jsonl


In [13]:
!shuf combine-malay-no-alignment-rag.jsonl > shuf-combine-malay-no-alignment-rag.jsonl

In [14]:
def generate_and_tokenize_prompt(row):
    texts = ['<s>']

    if 'function_call' in row:
        t = row['function_call']
        texts.append(f'\n[FUNCTIONCALL]\n{t}\n')

    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human.strip())
            outputs.append(bot.strip())
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    for u, a in zip(inputs, outputs):
        if not len(u.strip()) or not len(a.strip()):
            print(u, a)
            return
        texts.append(f'[INST] {u.strip()} [/INST] {a.strip()}</s> ')

    prompt = ''.join(texts)
    return {'text': prompt}

In [15]:
from tqdm import tqdm

data = []
with open('shuf-combine-malay-no-alignment-rag.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        if 'function_call' in l and not isinstance(l['function_call'], str):
            print(l['function_call'])
        if generate_and_tokenize_prompt(l) is not None:
            data.append(l)

72083it [00:01, 39991.33it/s]

176436it [00:04, 50762.61it/s]

 Ya, dalam era globalisasi dan kepelbagaian ekonomi, sistem kewangan Islam menawarkan potensi untuk mewujudkan kestabilan dan daya saing dalam pasaran sukuk. Walau bagaimanapun, cabarannya adalah meyakinkan pelabur, sama ada Muslim atau bukan Islam, tentang kekuatan dan keupayaan sistem kewangan Islam sebagai alternatif yang sejajar dengan sistem konvensional.

Dalam hal ini, perlu ada pendekatan yang komprehensif untuk mempromosikan sukuk kepada pelabur. Ini melibatkan pendidikan dan kesedaran yang lebih besar tentang prinsip syariah dan faedah ekonomi sukuk. Selain itu, ia juga penting untuk mengekalkan kualiti dan kepercayaan pasaran dengan memastikan terdapat rangka peraturan yang betul dan prosedur yang telus dalam penerbitan dan perdagangan sukuk.

Seiring dengan pertumbuhan pasaran sukuk global, peningkatan pilihan instrumen sukuk yang inovatif dan mempelbagaikan portfolio juga boleh menjadi faktor yang menyokong kejayaan sistem kewangan Islam. Oleh itu, usaha kerjasama daripada

222631it [00:04, 51314.71it/s]

 Terdapat beberapa faktor utama yang mempengaruhi perkembangan pasaran sukuk. Faktor-faktor ini termasuk pertimbangan pasaran dan komersial, serta kepentingan kesesuaian dengan syariah Islam. Kedua-dua faktor ini saling berkaitan dan tidak dapat dipisahkan.


...

2605336it [00:55, 50176.25it/s]

Dengan menggunakan hanya emotikon, cipta potret tokoh sejarah terkenal dari Malaysia. Sertakan juga unsur budaya atau warisan Malaysia. 


2615424it [00:55, 50100.48it/s]

?   . 


2775535it [00:59, 46604.95it/s]


In [16]:
len(data)

2775508

In [17]:
with open('shuf-combine-malay-no-alignment-rag.jsonl', 'w') as fopen:
    for d in data:
        fopen.write(f'{json.dumps(d)}\n')

In [18]:
!wc -l shuf-combine-malay-no-alignment-rag.jsonl

2775508 shuf-combine-malay-no-alignment-rag.jsonl


In [19]:
with open('shuf-combine-malay-no-alignment-rag.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        if 'function_call' in l and not isinstance(l['function_call'], str):
            print(l['function_call'])
        if generate_and_tokenize_prompt(l) is None:
            break

2775508it [00:55, 50264.77it/s]


In [20]:
!head -n 3 shuf-combine-malay-no-alignment-rag.jsonl

{"prompt_input": null, "input": "<manusia>: Periksa kesan pelbagai genre muzik ke atas otak manusia untuk menentukan sejauh mana muzik boleh mempengaruhi mood, kognisi dan tingkah laku pada peringkat biologi dan psikologi.\n<bot>: Muzik telah menjadi sebahagian daripada budaya manusia selama berabad-abad dan dipercayai mempunyai kesan yang mendalam terhadap emosi, kognisi dan tingkah laku manusia. Kajian yang dijalankan ke atas kesan pelbagai genre muzik pada otak telah menunjukkan bahawa muzik boleh mempengaruhi mood, kognisi dan tingkah laku pada tahap biologi dan psikologi.\n\nPenyelidikan telah menunjukkan bahawa mendengar muzik boleh meningkatkan mood seseorang dengan mengaktifkan pusat ganjaran otak, yang membawa kepada rembesan dopamin, neurotransmitter yang bertanggungjawab untuk mengawal mood dan motivasi. Dalam hal ini, genre muzik yang lebih ceria, bertenaga dan menggembirakan cenderung memberi kesan yang lebih ketara pada peningkatan mood berbanding genre yang lebih perlaha

In [21]:
!ls -lh shuf-combine-malay-no-alignment-rag.jsonl

-rw-r--r-- 1 husein husein 15G Jan  15 19:22 shuf-combine-malay-no-alignment-rag.jsonl


In [24]:
!rm -rf mosaic

In [25]:
from streaming import MDSWriter
from streaming import LocalDataset, StreamingDataset

columns = {
    'text': 'str',
}
hashes = 'sha1', 'xxh64'

with MDSWriter(out='mosaic', columns=columns, compression=None, hashes=hashes) as out:
    with open('shuf-combine-malay-no-alignment-rag.jsonl') as fopen:
        for l in tqdm(fopen):
            out.write({
                'text': l
            })

2775508it [02:02, 22689.32it/s]


In [26]:
!cp mosaic/* ~/ssd3/mosaic-chat-instructions-rag

In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('mesolitica/tinyllama-1.1b-4096-fpf')
tokenizer.pad_token = tokenizer.unk_token
tokenizer.add_bos_token = False
tokenizer.add_eos_token = False
tokenizer.padding_side = "right"
tokenizer.chat_template = "{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token}}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}"

In [28]:
def generate_and_tokenize_prompt(row):
    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except BaseException:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human)
            outputs.append(bot)
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    chat = []
    for input, output in zip(inputs, outputs):
        chat.extend([
            {'role': 'user', 'content': input.strip()},
            {'role': 'assistant', 'content': output.strip()},
        ])
    prompt = tokenizer.apply_chat_template(chat, tokenize=False)
    return {'text': prompt}

In [29]:
dataset = LocalDataset(local='mosaic')
row = json.loads(dataset[0]['text'])
element = generate_and_tokenize_prompt(row)
outputs = tokenizer(
    element['text'],
    truncation=True,
    padding=False,
    max_length=16384,
    return_overflowing_tokens=False,
    return_length=False,
)

In [30]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='shuf-combine-malay-no-alignment-rag.jsonl',
    path_in_repo='shuf-combine-malay-no-alignment-rag.jsonl',
    repo_id='mesolitica/instructions-dataset',
    repo_type='dataset',
)

shuf-combine-malay-no-alignment-rag.jsonl:   0%|          | 0.00/15.6G [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/instructions-dataset/blob/main/shuf-combine-malay-no-alignment-rag.jsonl'